In [1]:
from __future__ import print_function

import random
import numpy as np
from lib.data_utils import load_CIFAR10
import matplotlib.pyplot as plt


%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading extenrnal modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

np.random.seed(1)

In [3]:
def get_CIFAR10_data(num_training=49000, num_validation=1000, num_test=1000, num_dev=500):
    """
    Load the CIFAR-10 dataset from disk and perform preprocessing to prepare
    it for the linear classifier. These are the same steps as we used for the
    SVM, but condensed to a single function.  
    """
    # Load the raw CIFAR-10 data
    cifar10_dir = 'lib/datasets/cifar-10-batches-py'
    X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)
    
    # subsample the data
    mask = list(range(num_training, num_training + num_validation))
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = list(range(num_training))
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = list(range(num_test))
    X_test = X_test[mask]
    y_test = y_test[mask]
    mask = np.random.choice(num_training, num_dev, replace=False)
    X_dev = X_train[mask]
    y_dev = y_train[mask]
    
    # Preprocessing: reshape the image data into rows
    X_train = np.reshape(X_train, (X_train.shape[0], -1))
    X_val = np.reshape(X_val, (X_val.shape[0], -1))
    X_test = np.reshape(X_test, (X_test.shape[0], -1))
    X_dev = np.reshape(X_dev, (X_dev.shape[0], -1))
    
    # Normalize the data: subtract the mean image
    mean_image = np.mean(X_train, axis = 0)
    X_train -= mean_image
    X_val -= mean_image
    X_test -= mean_image
    X_dev -= mean_image
    
    # add bias dimension and transform into columns
    X_train = np.hstack([X_train, np.ones((X_train.shape[0], 1))])
    X_val = np.hstack([X_val, np.ones((X_val.shape[0], 1))])
    X_test = np.hstack([X_test, np.ones((X_test.shape[0], 1))])
    X_dev = np.hstack([X_dev, np.ones((X_dev.shape[0], 1))])
    
    return X_train, y_train, X_val, y_val, X_test, y_test, X_dev, y_dev


# Invoke the above function to get our data.
X_train, y_train, X_val, y_val, X_test, y_test, X_dev, y_dev = get_CIFAR10_data()
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)
print('dev data shape: ', X_dev.shape)
print('dev labels shape: ', y_dev.shape)

Train data shape:  (49000, 3073)
Train labels shape:  (49000,)
Validation data shape:  (1000, 3073)
Validation labels shape:  (1000,)
Test data shape:  (1000, 3073)
Test labels shape:  (1000,)
dev data shape:  (500, 3073)
dev labels shape:  (500,)


In [4]:
y_dev

array([7, 9, 9, 4, 6, 5, 3, 2, 5, 4, 1, 6, 3, 7, 2, 9, 0, 9, 3, 6, 1, 4,
       6, 4, 4, 1, 6, 7, 5, 6, 6, 6, 0, 8, 0, 8, 6, 9, 5, 5, 5, 5, 2, 5,
       2, 0, 6, 8, 3, 9, 7, 3, 6, 4, 9, 0, 8, 6, 0, 9, 5, 1, 3, 7, 9, 1,
       2, 4, 5, 0, 5, 7, 6, 7, 1, 9, 0, 5, 7, 3, 1, 8, 6, 9, 4, 3, 6, 8,
       9, 4, 3, 4, 6, 5, 7, 5, 3, 6, 2, 7, 0, 2, 5, 6, 8, 1, 8, 7, 4, 2,
       4, 6, 3, 0, 9, 4, 8, 0, 4, 7, 2, 0, 7, 1, 8, 5, 7, 2, 2, 0, 4, 0,
       3, 8, 1, 8, 3, 9, 8, 1, 5, 9, 4, 6, 8, 4, 2, 4, 9, 3, 1, 0, 9, 4,
       3, 6, 2, 1, 8, 8, 8, 2, 6, 6, 9, 8, 0, 6, 8, 5, 9, 4, 8, 3, 8, 1,
       9, 7, 1, 7, 0, 6, 6, 0, 5, 4, 7, 4, 3, 9, 3, 8, 5, 0, 8, 4, 7, 3,
       5, 9, 1, 0, 1, 0, 1, 7, 6, 2, 4, 2, 4, 2, 2, 2, 9, 2, 8, 6, 0, 8,
       2, 6, 4, 8, 3, 7, 2, 1, 5, 3, 0, 4, 0, 6, 1, 6, 0, 5, 6, 7, 5, 0,
       9, 4, 7, 5, 7, 7, 1, 4, 9, 7, 2, 6, 7, 6, 4, 1, 0, 9, 8, 7, 5, 9,
       9, 2, 1, 8, 4, 9, 2, 2, 4, 0, 6, 0, 2, 6, 9, 1, 6, 0, 2, 7, 7, 0,
       1, 2, 2, 8, 5, 7, 4, 6, 0, 1, 3, 3, 9, 3, 9,

In [49]:
from random import shuffle
from past.builtins import xrange

def softmax_loss_naive(W, X, y, reg):
    """
    Softmax loss function, naive implementation (with loops)

    Inputs have dimension D, there are C classes, and we operate on minibatches
    of N examples.

    Inputs:
    - W: A numpy array of shape (D, C) containing weights.
    - X: A numpy array of shape (N, D) containing a minibatch of data.
    - y: A numpy array of shape (N,) containing training labels; y[i] = c means
    that X[i] has label c, where 0 <= c < C.
    - reg: (float) regularization strength

    Returns a tuple of:
    - loss as single float
    - gradient with respect to weights W; an array of same shape as W
    """
    # Initialize the loss and gradient to zero.
    loss = 0.0
    dW = np.zeros_like(W)

    #############################################################################
    # TODO: Compute the softmax loss and its gradient using explicit loops.     #
    # Store the loss in loss and the gradient in dW. If you are not careful     #
    # here, it is easy to run into numeric instability. So use stable version   #
    # of softmax. Don't forget the regularization!                              #
    #############################################################################
    num_train = X.shape[0]
    summ = 0
    N = len(dW) * len(dW[0])

    for i in xrange(num_train):
        scores = X[i].dot(W)
        exp = np.exp(scores)
        prob = exp / np.sum(exp)
        Li = -1 * np.log(prob)
        loss += np.sum(Li) / len(Li) + np.sum(W**2) * reg
        dW[i] = prob
    loss = loss / num_train

    #############################################################################
    #                          END OF YOUR CODE                                 #
    #############################################################################

    return loss, dW

In [53]:
scores = X_dev[0].dot(W)
exp = np.exp(scores)
prob = exp / np.sum(exp)
Li = -1 * np.log(prob)
L = np.sum(Li) / len(Li)
L
print(len(W[0]), len(prob))
print(len(W),X_dev.shape[0])

10 10
3073 500


In [50]:
W = np.random.randn(3073, 10) * 0.0001
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 0.0)
print(grad)
# As a rough sanity check, our loss should be something close to -log(0.1).
print('loss: %f' % loss)
print('sanity check: %f' % (-np.log(0.1)))

[[0.10869767 0.07757135 0.08899299 ... 0.10188429 0.07427537 0.06126097]
 [0.1216166  0.09631882 0.05428237 ... 0.08060439 0.15158943 0.11641949]
 [0.11509727 0.12218299 0.07162603 ... 0.17157002 0.14942395 0.07336038]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]
loss: 2.354191
sanity check: 2.302585


In [31]:
print(len(W))
print(W[0])
len(X_dev[0])

3073
[-4.13696069e-05  2.51860408e-04  2.34100311e-05 -4.07824769e-06
 -8.25956828e-05  1.17850723e-04  1.30201228e-04  9.05612037e-05
  6.17478435e-05 -8.19747596e-06]


3073

In [38]:
np.array([1,2,3,4]) * np.array([5,6,7,8])

array([ 5, 12, 21, 32])

In [37]:
num_train = X_dev.shape[0]
num_train

500

In [6]:
tmp = np.tile(np.array([[1],[2],[3],[4]]), 10)
1 / np.sum(tmp, axis=1)
tmp

array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
       [3, 3, 3, 3, 3, 3, 3, 3, 3, 3],
       [4, 4, 4, 4, 4, 4, 4, 4, 4, 4]])